<a href="https://colab.research.google.com/github/Nhrot22230/DeepSampler/blob/feat%2Frefactor-data-pipeline/refactor-data-pipeline/notebooks/exploratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Nhrot22230/DeepSampler
!cd DeepSampler && git pull && git checkout feat/refactor-data-pipeline

Cloning into 'DeepSampler'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 597 (delta 76), reused 130 (delta 43), pack-reused 421 (from 1)
Receiving objects: 100% (597/597), 4.23 MiB | 12.18 MiB/s, done.
Resolving deltas: 100% (298/298), done.


In [ ]:
!cd DeepSampler && make init

In [16]:
import os
import sys

# Suponiendo que clonaste el repositorio en /content/DeepSampler
project_root = os.path.join(os.getcwd(), "DeepSampler")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [22]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import matplotlib.pyplot as plt
from src.pipelines.data import musdb_pipeline
from src.pipelines.train import train_pipeline
from src.pipelines.inference import infer_pipeline
from src.models.deep_sampler import DeepSampler
from src.models.scunet import SCUNet
from src.utils.training.loss import MultiSourceLoss, VGGFeatureLoss, MultiScaleLoss
import numpy as np

plt.rcParams["figure.figsize"] = [20, 6]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
nfft = 2048
hop_length = 512
window = torch.hann_window(nfft)
chunk_seconds = 2
overlap = 0
sr = 44100

data_root = os.path.join(project_root, "data")
musdb_root = os.path.join(data_root, "musdb18hq")

if not os.path.exists(data_root):
    raise FileNotFoundError(
        "No se encontró la carpeta data, por favor ejecute el script download_data.sh antes de ejecutar este script."
    )

In [ ]:
# train_dataset = musdb_pipeline(
#     musdb_path=os.path.join(musdb_root, "train"),
#     nfft=nfft,
#     hop_length=hop_length,
#     window=window,
#     chunk_seconds=chunk_seconds,
#     overlap=overlap,
#     sample_rate=sr,
# )
# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=16, shuffle=True
# )

In [24]:
test_dataset = musdb_pipeline(
    musdb_path=os.path.join(musdb_root, "test"),
    nfft=nfft,
    hop_length=hop_length,
    window=window,
    chunk_seconds=chunk_seconds,
    overlap=overlap,
    sample_rate=sr,
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=16, shuffle=False
)

Procesando pistas: 100%|██████████| 50/50 [02:38<00:00,  3.17s/it]


In [25]:
deep_sampler = DeepSampler(
    input_channels=1,
    output_channels=4,
    base_channels=32,
    depth=5,
    dropout=0.1,
    transformer_heads=4,
    transformer_layers=4,
)
optimizer = optim.Adam(deep_sampler.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
criterion = MultiSourceLoss(
    weights=[1, 1, 1, 1],
    distance="l1",
)

factor = 1
epochs = 5 * factor
p1_epochs = 2 * factor

In [ ]:
train_pipeline(
    model=deep_sampler,
    dataloader=test_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    total_epochs=epochs,
    phase1_epochs=p1_epochs,
    device=device,
)

Epoch 1/5:   0%|          | 0/389 [00:00<?, ?batch/s]

In [ ]:
test_folders = os.listdir(os.path.join(musdb_root, "test"))
random_folder = np.random.choice(test_folders)

In [ ]:
sources = infer_pipeline(
    model=deep_sampler,
    mixture_path=os.path.join(musdb_root, "test", random_folder, "mixture.wav"),
    sample_rate=44100,
    chunk_seconds=chunk_seconds,
    overlap=overlap,
    n_fft=nfft,
    hop_length=hop_length,
    window=window,
    device=device,
)